# 02 – Modeling & Comparison

In [ ]:
import pandas as pd, numpy as np
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("../data/raw/sample_AAPL.csv", parse_dates=["Date"])
split = int(len(df)*0.8)
train, test = df[:split], df[split:]

# SARIMA
sar = SARIMAX(train.set_index("Date")["Close"], order=(1,1,1), seasonal_order=(1,1,1,12)).fit(disp=False)
s_fc = sar.get_forecast(steps=len(test)).predicted_mean
sar_rmse = mean_squared_error(test["Close"], s_fc, squared=False)

# Prophet
d = train[["Date","Close"]].rename(columns={"Date":"ds","Close":"y"})
m = Prophet(daily_seasonality=True).fit(d)
fut = m.make_future_dataframe(periods=len(test))
p_fc = m.predict(fut).set_index("ds").loc[test["Date"]]["yhat"]
p_rmse = mean_squared_error(test["Close"], p_fc, squared=False)

# LSTM
look_back = 60
series = train[["Date","Close"]].set_index("Date").values
scaler = MinMaxScaler((0,1))
scaled = scaler.fit_transform(series)
X,y=[],[]
for i in range(look_back, len(scaled)):
    X.append(scaled[i-look_back:i,0]); y.append(scaled[i,0])
X = np.array(X).reshape(-1,look_back,1); y = np.array(y)
model = Sequential([LSTM(50, return_sequences=True, input_shape=(look_back,1)), LSTM(50), Dense(1)])
model.compile(optimizer="adam", loss="mse")
model.fit(X, y, epochs=3, batch_size=32, verbose=0)
seq = scaled[-look_back:].reshape(1,look_back,1)
preds=[]
for _ in range(len(test)):
    p = model.predict(seq, verbose=0)[0][0]; preds.append(p)
    seq = np.append(seq[:,1:,:], [[[p]]], axis=1)
l_fc = scaler.inverse_transform(np.array(preds).reshape(-1,1)).flatten()
lstm_rmse = mean_squared_error(test["Close"].values, l_fc, squared=False)

pd.DataFrame({"Model":["SARIMA","Prophet","LSTM"],"RMSE":[sar_rmse,p_rmse,lstm_rmse]})